# Finetune Pix2Struct model on Pix2Code HTML dataset

## Setup Envirnoment

In [ ]:
!pip install --upgrade git+https://github.com/huggingface/transformers

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-qlwvu_j4
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-qlwvu_j4
  Resolved https://github.com/huggingface/transformers to commit 5936c8c57ccb2bda3b3f28856a7ef992c5c9f451
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 74.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 75.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 29.6 MB/s eta 0:00:00
  Created wheel for transformers: filename=transformers-4.34.0.dev0-py3-none-any.whl size=7708600 sha256=64d9937fc2987f45b41960b5476e20347ddb52d6c1a9344cb8269cf37dd7e396
  Stored in directory: /tmp/pip-ephem-wheel-cache-02o5t_h

In [ ]:
!pip install -q wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.0/190.0 kB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.8/224.8 kB 25.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.7 MB/s eta 0:00:00


## Import necessary libraries

In [ ]:
from google.colab import drive
import os
import zipfile
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import re
from transformers import Pix2StructForConditionalGeneration, AutoProcessor
import torch
from torch.nn import functional as F
from transformers.optimization import Adafactor, get_cosine_schedule_with_warmup
from pathlib import Path
from nltk import edit_distance
import numpy as np
import wandb
from tqdm import tqdm
from nltk.translate.bleu_score import corpus_bleu, sentence_bleu, SmoothingFunction
from torch.utils.data import random_split
import random

## Define variables and parameters

In [ ]:
G_DRIVE_FOLDER_DATASET = '/content/drive/MyDrive/Datasets/'
G_DRIVE_FOLDER_CHECKPOINTS = '/content/drive/MyDrive/Checkpoints/'
DATASET_NAME = 'pix2code_web_with_html'
ZIP_NAME = DATASET_NAME + '.zip'
DESTINATION_FOLDER= '/content/data/'
DATASET_FOLDER = DESTINATION_FOLDER + 'web_with_html/' # unzipped name

EXPERIMENT_NAME = "Pix2Struct_Pix2Code_HTML"

MAX_SENTENCE_LEN = 1024

MAX_PATCHES = 1024

DEBUG = False
VERBOSE = True

BATCH_SIZE = 4
NUM_WARMUP_STEPS = 500
MAX_EPOCHS = 10
LR = 1e-4
CHECK_VAL_EVERY_N_EPOCH = 5
GRADIENT_CLIP_VAL = 1.0
ACCUMULATE_GRAD_BATCHES = 8 / BATCH_SIZE

TRAIN_SET_PERCENTAGE = 0.89
VALID_SET_PERCENTAGE = 0.01
# TEST_SET_PERCENTAGE is 1 - TRAIN_SET_PERCENTAGE - VALID_SET_PERCENTAGE # Use 1000 for test


RANDOM_SEED = 100

LOAD_FROM_CHECKPOINT = True
LAST_CHECKPOINT_NAME = "FINAL_Pix2Struct_Pix2Code_epoch[19]_bleu[0.98].pth"

In [ ]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


## Load Pix2Code Dataset

### Mount Google Drive

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


### Import zip file from Google Drive

In [ ]:
os.makedirs(DESTINATION_FOLDER, exist_ok=True)

with zipfile.ZipFile(G_DRIVE_FOLDER_DATASET + ZIP_NAME, "r") as zf:
  zf.extractall(DESTINATION_FOLDER)

## Load Model and Processor

In [ ]:
repo_id = "google/pix2struct-base"

processor = AutoProcessor.from_pretrained(repo_id)
model = Pix2StructForConditionalGeneration.from_pretrained(repo_id, is_encoder_decoder=True)

Loading the tokenizer from the `special_tokens_map.json` and the `added_tokens.json` will be removed in `transformers 5`,  it is kept for forward compatibility, but it is recommended to update your `tokenizer_config.json` by uploading it again. You will see the new `added_tokens_decoder` attribute that will store the relevant information.


In [ ]:
if LOAD_FROM_CHECKPOINT:
    print("Loading model from checkpoint:", LAST_CHECKPOINT_NAME)
    checkpoint = torch.load(G_DRIVE_FOLDER_CHECKPOINTS + LAST_CHECKPOINT_NAME)

    # Load only model weights from training on Pix2Code guis
    model.load_state_dict(checkpoint)


Loading model from checkpoint: FINAL_Pix2Struct_Pix2Code_epoch[19]_bleu[0.98].pth


## Create Dataset class

In [ ]:
def preprocess_html_file(html_text):
    text_without_header = re.sub(r'<header>.*?</header>', '', html_text, flags=re.DOTALL)
    text_without_footer = re.sub(r'<footer class="footer">.*?</footer>', '', text_without_header, flags=re.DOTALL)
    text_without_script = re.sub(r'<script .*?</script>', '', text_without_footer, flags=re.DOTALL)
    text_without_linebreaks = text_without_script.replace('\n', ' ')
    text_without_multiple_spaces = re.sub(r'\s+', ' ', text_without_linebreaks)
    return text_without_multiple_spaces

### Filter files with less tokens than 1024 and add new unknown tokens

In [ ]:
# Get a list of all files in root_dir
files = os.listdir(DATASET_FOLDER)

# Find only html files
all_html_files = [file for file in files if file.endswith('.html')]

In [ ]:
# Find max length
max_length = 0

bigger_than_1024 = 0
lower_than_1024 = 0

html_files_filtered = []

tokens_to_add = set()

for html_file_path in all_html_files:
    with open(DATASET_FOLDER + "/" + html_file_path, "r") as reader:
        preprocessed_text = preprocess_html_file(reader.read())
        splitted_text = processor.tokenizer(preprocessed_text).tokens()
        if len(splitted_text) > 1024:
            bigger_than_1024 += 1
        else:
            lower_than_1024 += 1
            html_files_filtered.append(html_file_path)
            tokens_to_add = tokens_to_add.union(set(splitted_text))

print("bigger_than_1024= ", bigger_than_1024)
print("lower_than_1024= ", lower_than_1024)

newly_added_num = processor.tokenizer.add_tokens(list(tokens_to_add))
print(f"Number of new tokens = {newly_added_num}")

# Resize the model's token embeddings if there are new tokens
if newly_added_num > 0:
    model.decoder.resize_token_embeddings(len(processor.tokenizer))

You are resizing the embedding layer without providing a `pad_to_multiple_of` parameter. This means that the new embedding dimension will be 50344. This might induce some performance reduction as *Tensor Cores* will not be available. For more details about this, or help on choosing the correct value for resizing, refer to this guide: https://docs.nvidia.com/deeplearning/performance/dl-performance-matrix-multiplication/index.html#requirements-tc


bigger_than_1024=  9
lower_than_1024=  1733
Number of new tokens = 5942


In [ ]:
print(len(html_files_filtered))

1733


In [ ]:
random.seed(RANDOM_SEED)

# Use the same seed, so that parts remain the same
random.shuffle(html_files_filtered)

train_len = int(TRAIN_SET_PERCENTAGE * len(html_files_filtered))
valid_len = int(VALID_SET_PERCENTAGE * len(html_files_filtered))

train_paths = html_files_filtered[:train_len]
valid_paths = html_files_filtered[train_len:train_len+valid_len]
test_paths = html_files_filtered[train_len+valid_len:]

print(f"TRAIN_SET size = {len(train_paths)}")
print(f"VALID_SET size = {len(valid_paths)}")
print(f"TEST_SET size = {len(test_paths)}")

TRAIN_SET size = 1542
VALID_SET size = 17
TEST_SET size = 174


In [ ]:
class Pix2CodeDataset(Dataset):
    def __init__(self, root_dir, transform, text_files_paths):

        self.root_dir = root_dir
        self.transform = transform
        self.text_files_paths = text_files_paths

        self.max_patches = MAX_PATCHES
        self.max_length = MAX_SENTENCE_LEN
        self.ignore_id = -100

        self.encodings = []

        for text_file in tqdm(text_files_paths):
            image_file = text_file.replace('.html', '.png')

            # Directly process the text files, and save them in the ram
            # Do the same also for images, if there is enough space in memory
            text_file_path = os.path.join(root_dir, text_file)
            image_file_path = os.path.join(root_dir, image_file)

            # Load image
            image = Image.open(image_file_path).convert('RGB')

            if DEBUG:
                image.show()

            if self.transform:
                image = self.transform(image)

            encoding = processor(images=image, max_patches=self.max_patches, return_tensors="pt")
            encoding = {k:v.squeeze() for k,v in encoding.items()}

            # Load text
            with open(text_file_path, 'r') as f:
                text = f.read()
                text_cleaned = preprocess_html_file(text)

            if DEBUG:
              print("text:")
              print(text)
              print("\n\n\ntext_cleaned:")
              print(text_cleaned)

            input_ids = processor.tokenizer(
                text_cleaned,
                max_length=self.max_length,
                padding="max_length",
                truncation=True,
                return_tensors="pt",
            ).input_ids

            labels = input_ids.squeeze().clone()
            labels[labels == processor.tokenizer.pad_token_id] = self.ignore_id  # model doesn't need to predict pad token

            encoding["labels"] = labels

            # For each sample save directly the encoding of both text and image
            self.encodings.append(encoding)

    def __len__(self):
        return len(self.encodings)

    def __getitem__(self, idx):
        return self.encodings[idx]

In [ ]:
# Transformations for the image
transform = transforms.Compose([
    transforms.ToTensor(),  # convert PIL Image to PyTorch Tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # normalize for pretrained models
])

# Instantiate the CustomDataset
train_dataset = Pix2CodeDataset(DATASET_FOLDER, transform, train_paths)
val_dataset = Pix2CodeDataset(DATASET_FOLDER, transform, valid_paths)

# Use DataLoader for batching and shuffling
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=10, shuffle=False) # Use 10 as batch for testing

100%|██████████| 17/17 [00:00<00:00, 17.15it/s]


In [ ]:
print(f"train_dataloader size = {len(train_dataloader)}")
print(f"val_dataloader size = {len(val_dataloader)}")

train_dataloader size = 386
val_dataloader size = 2


In [ ]:
batch = next(iter(train_dataloader))
encoding = batch

In [ ]:
encoding

{'flattened_patches': tensor([[[ 1.0000,  1.0000, -0.0160,  ..., -0.0160,  0.2897,  0.6495],
          [ 1.0000,  2.0000, -0.0160,  ..., -5.9606, -5.7876, -5.4008],
          [ 1.0000,  3.0000, -0.0160,  ..., -5.6998, -4.7901, -3.5922],
          ...,
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],
 
         [[ 1.0000,  1.0000, -0.0179,  ..., -0.0179,  0.2891,  0.6503],
          [ 1.0000,  2.0000, -0.0179,  ..., -5.9860, -5.8122, -5.4239],
          [ 1.0000,  3.0000, -0.0179,  ..., -5.9705, -5.7504, -5.3106],
          ...,
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],
 
         [[ 1.0000,  1.0000, -0.1284,  ..., -0.1284,  0.2294,  0.6505],
       

In [ ]:
encoding["flattened_patches"][0]

tensor([[ 1.0000,  1.0000, -0.0160,  ..., -0.0160,  0.2897,  0.6495],
        [ 1.0000,  2.0000, -0.0160,  ..., -5.9606, -5.7876, -5.4008],
        [ 1.0000,  3.0000, -0.0160,  ..., -5.6998, -4.7901, -3.5922],
        ...,
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]])

In [ ]:
encoding["labels"]

tensor([[ 589,  812,  371,  ..., -100, -100, -100],
        [ 589,  812,  371,  ..., -100, -100, -100],
        [ 589,  812,  371,  ..., -100, -100, -100],
        [ 589,  812,  371,  ..., -100, -100, -100]])

In [ ]:
encoding["labels"][0]

tensor([ 589,  812,  371,  ..., -100, -100, -100])

In [ ]:
labels_list = encoding["labels"][0].tolist()

# Filter out the -100 values
filtered_labels = [token for token in labels_list if token != -100]

# Decode the cleaned list of tokens
decoded_text_example = processor.tokenizer.batch_decode([filtered_labels], skip_special_tokens=True)[0]


In [ ]:
decoded_text_example

'<html> <body> <main class="container"> <div class="header clearfix"> <nav> <ul class="nav nav-pills pull-left"> <li><a href="#">Ihf Fbqadm</a></li> <li><a href="#">Ir Rtnjvmr</a></li> <li><a href="#">Oa Apexnpz</a></li> <li><a href="#">Rg Gynxhke</a></li> <li class="active"><a href="#">Zkcdau Ugj</a></li> </ul> </nav> </div> <div class="row"><div class="col-lg-6"> <h4>Muiaw</h4><p>raufxdpavw ketenu iceeowizvdnban ybhnzavl alnbbxvzbbwwa</p> <a class="btn btn-warning" href="#" role="button">Vaz Zamqsu</a> </div> <div class="col-lg-6"> <h4>Aykqr</h4><p>koafucqsatanmheunqcoxhfvtsswdmyuhfrviidqinqwgfey yzwpgrk</p> <a class="btn btn-danger" href="#" role="button">Kjv Vttfnv</a> </div> </div> <div class="row"><div class="col-lg-12"> <h4>Pmwli</h4><p>ptjq yqr fxb eq htyreorzkcg cdl qgnskckkmzvngeduvroyj ja</p> <a class="btn btn-success" href="#" role="button">Hgq Qgvdrf</a> </div> </div> <div class="row"><div class="col-lg-3"> <h4>Qudxz</h4><p>oykfawqixhgfrqiyczrpgwja gt bgv osyyspgbo moi a b

In [ ]:
for k,v in encoding.items():
    print(k,v.shape)

flattened_patches torch.Size([4, 1024, 770])
attention_mask torch.Size([4, 1024])
labels torch.Size([4, 1024])


## Training

In [ ]:
START_TOKEN_ID = PAD_TOKEN_ID = processor.tokenizer.pad_token_id

In [ ]:
def move_to_device(data):
    if isinstance(data, (list,tuple)):
        return [move_to_device(x) for x in data]
    elif isinstance(data, dict):
        return {k: move_to_device(v) for k, v in data.items()}
    elif isinstance(data, torch.Tensor):
        return data.to(DEVICE)
    else:
        return data

### Main training function

In [ ]:
def train_model(config, processor, model, train_dataloader, val_dataloader):
    # Extract configuration values
    lr = config.get("lr")
    max_epochs = config.get("max_epochs")
    num_warmup_steps = config.get("num_warmup_steps")

    model.to(DEVICE)

    optimizer = Adafactor(model.parameters(), scale_parameter=False, relative_step=False, lr=lr, weight_decay=1e-05)

    # Use total steps (i.e., max_epochs * length_of_train_data)
    total_steps = max_epochs * len(train_dataloader)
    scheduler = get_cosine_schedule_with_warmup(optimizer,
                                                num_warmup_steps=num_warmup_steps,
                                                num_training_steps=total_steps)

    global_step = 0  # to keep track of total steps
    epoch_start = 0

    wandb.init(project="Pix2Struct", name="run-" + EXPERIMENT_NAME, config=config)

    #epoch_last = epoch_start + max_epochs - 1
    epoch_last = max_epochs - 1
    #for epoch in range(epoch_start, epoch_start + max_epochs):
    for epoch in range(epoch_start, max_epochs):
        global_step, moving_avg_loss = training_loop(epoch, train_dataloader, model, config, optimizer, scheduler, global_step, epoch_last)

        if config.get("verbose", False):
            print(f"Moving Avg Loss: {moving_avg_loss:.3f}")

        wandb.log({"moving_avg_loss": moving_avg_loss, **{f'lr_{i}': param_group['lr'] for i, param_group in enumerate(optimizer.param_groups)}})

        # Save the model after each validation step
        save_checkpoint(model, optimizer, scheduler, epoch, global_step, wandb.run.id, EXPERIMENT_NAME, G_DRIVE_FOLDER_CHECKPOINTS)

        #if epoch == 0 + epoch_start or epoch == epoch_last or (epoch + 1) % config.get("check_val_every_n_epoch") == 0:
        if epoch == 0 or epoch == epoch_last or (epoch + 1) % config.get("check_val_every_n_epoch") == 0:
            avg_bleu_score = testing_loop(val_dataloader, model, processor, config, f"Epoch {epoch}/{epoch_last} - valid loop")

            if config.get("verbose", False):
                print(f" Avg Bleu Score: {avg_bleu_score:.2f}")

            wandb.log({"bleu": avg_bleu_score})

    wandb.finish()

In [ ]:
def training_loop(epoch, train_dataloader, model, config, optimizer, scheduler, global_step, epoch_last):
    model.train()
    train_loop = tqdm(enumerate(train_dataloader), total=len(train_dataloader), desc=f"Epoch {epoch}/{epoch_last} - train loop")

    # Extract configuration values
    accumulate_grad_batches = config.get('accumulate_grad_batches', 1)
    gradient_clip_val = config.get("gradient_clip_val")

    moving_avg_loss = 0
    alpha = 0.1 # Smoothing factor

    for step, batch in train_loop:
        encoding = move_to_device(batch)
        labels, flattened_patches, attention_mask = encoding["labels"], encoding["flattened_patches"], encoding["attention_mask"]

        outputs = model(labels=labels, flattened_patches=flattened_patches, attention_mask=attention_mask)
        loss = outputs.loss
        loss.backward()

        if global_step % accumulate_grad_batches == 0 or step == len(train_dataloader) - 1:
            if gradient_clip_val:
                torch.nn.utils.clip_grad_norm_(model.parameters(), gradient_clip_val)
            optimizer.step()
            optimizer.zero_grad()

        # Update the progress bar
        train_loop.set_postfix({'loss': loss.item()}, refresh=True)

        scheduler.step()
        global_step += 1

        # Update the moving average loss
        moving_avg_loss = loss.item() if moving_avg_loss == 0 else alpha * loss.item() + (1 - alpha) * moving_avg_loss

        # Log Loss after each step
        wandb.log({"loss": loss.item()})

    return global_step, moving_avg_loss

In [ ]:
def testing_loop(testing_dataloader, model, processor, config, description):
    model.eval()
    bleu_scores = []

    with torch.no_grad():
        test_loop = tqdm(enumerate(testing_dataloader), total=len(testing_dataloader), desc=description)
        for i, batch in test_loop:
            encoding = move_to_device(batch)
            labels, flattened_patches, attention_mask = encoding["labels"], encoding["flattened_patches"], encoding["attention_mask"]

            outputs = model.generate(flattened_patches=flattened_patches, attention_mask=attention_mask, max_new_tokens=MAX_SENTENCE_LEN)

            predictions = processor.tokenizer.batch_decode(outputs, skip_special_tokens=True)

            labels[labels == -100] = 0
            answers = processor.tokenizer.batch_decode(labels, skip_special_tokens=True)

            bleu_scores += [corpus_bleu([[answer]], [pred], smoothing_function=SmoothingFunction().method4) for pred, answer in zip(predictions, answers)]

            avg_bleu_score = np.mean(bleu_scores)
            test_loop.set_postfix(bleu_score=avg_bleu_score)

            if config.get("verbose", False):
                for pred, answer, bleu_score in zip(predictions, answers, bleu_scores):
                    tqdm.write(f"\nPrediction: {pred}\n    Answer: {answer}\n      Bleu: {bleu_score:.2f}")

    return avg_bleu_score


In [ ]:
def save_checkpoint(model, optimizer, scheduler, epoch, global_step, wandb_run_id, experiment_name, folder_path):
    checkpoint = {
        "model_state_dict": model.state_dict(),
        "optimizer_state_dict": optimizer.state_dict(),
        "scheduler_state_dict": scheduler.state_dict(),
        "epoch": epoch,
        "global_step": global_step,
        'wandb_run_id': wandb_run_id
    }
    model_name = f"{experiment_name}_epoch[{epoch}].pth"
    torch.save(checkpoint, folder_path + model_name)


In [ ]:
config = {
          "batch_size": BATCH_SIZE,
          "num_warmup_steps": NUM_WARMUP_STEPS,
          "max_epochs": MAX_EPOCHS,
          "lr": LR,
          "check_val_every_n_epoch": CHECK_VAL_EVERY_N_EPOCH,
          "gradient_clip_val": GRADIENT_CLIP_VAL,
          "accumulate_grad_batches": ACCUMULATE_GRAD_BATCHES,
          "verbose": VERBOSE,
}

In [ ]:
def validate_config(config):
    # Check required keys
    required_keys = [
        "batch_size",
        "num_warmup_steps",
        "max_epochs",
        "lr",
        "check_val_every_n_epoch",
        "gradient_clip_val",
        "accumulate_grad_batches",
        "verbose"
    ]
    for key in required_keys:
        if key not in config:
            raise ValueError(f"Key '{key}' must be present in the configuration.")

    # Check that values are in expected ranges
    if config["batch_size"] <= 0:
        raise ValueError("batch_size must be positive.")
    if config["num_warmup_steps"] < 0:
        raise ValueError("num_warmup_steps must be non-negative.")
    if config["max_epochs"] <= 0:
        raise ValueError("max_epochs must be positive.")
    if config["lr"] <= 0:
        raise ValueError("Learning rate must be positive.")
    if config["check_val_every_n_epoch"] <= 0:
        raise ValueError("check_val_every_n_epoch must be positive.")
    if config["gradient_clip_val"] < 0:
        raise ValueError("gradient_clip_val must be non-negative.")
    if config["accumulate_grad_batches"] <= 0:
        raise ValueError("accumulate_grad_batches must be positive.")
    if not isinstance(config["verbose"], bool):
        raise ValueError("verbose must be a boolean value.")


In [ ]:
validate_config(config)
print(config)

{'batch_size': 4, 'num_warmup_steps': 500, 'max_epochs': 10, 'lr': 0.0001, 'check_val_every_n_epoch': 5, 'gradient_clip_val': 1.0, 'accumulate_grad_batches': 2.0, 'verbose': True}


In [ ]:
train_model(config, processor, model, train_dataloader, val_dataloader)

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch 0/9 - train loop: 100%|██████████| 386/386 [05:00<00:00,  1.28it/s, loss=2.13]


Moving Avg Loss: 2.251


Epoch 0/9 - valid loop:  50%|█████     | 1/2 [03:01<03:01, 181.67s/it, bleu_score=0.179]

...

Epoch 9/9 - valid loop:  50%|█████     | 1/2 [02:57<02:57, 177.18s/it, bleu_score=0.206]


Prediction: <html> <body> <main class="container"> <div class="header clearfix"> <nav> <ul class="nav nav-pills pull-left"> <li><a href="#">Xs Sssssss</a></li> <li><a href="#">Xs Sssssss</a></li> <li class="active"><a href="#">Xs Sssssss</a></li> <li><a href="#">Xs Sssssss</a></li> <li><a href="#">Xs Sssssss</a></li> </ul> </nav> </div> <div class="row"><div class="col-lg-12"> <h4>Eurzh</h4><p>s s s s s s s s s s s s s s s ss ss ss ss ss ss ss s s ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss s

Epoch 9/9 - valid loop: 100%|██████████| 2/2 [05:04<00:00, 152.45s/it, bleu_score=0.214]


Prediction: <html> <body> <main class="container"> <div class="header clearfix"> <nav> <ul class="nav nav-pills pull-left"> <li><a href="#">Xs Ssssssss</a></li> <li><a href="#">Xs Ssssssss</a></li> <li><a href="#">Xs Ssssssss</a></li> <li class="active"><a href="#">Xs Ssssssss</a></li> </ul> </nav> </div> <div class="row"><div class="col-lg-3"> <h4>Sbmav</h4><p>ss s s s s s s s s s s s s s ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss ss sss ss

bleu,▁█▄
loss,█▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
lr_0,▆█▇▇▅▄▃▂▁▁
moving_avg_loss,█▄▂▂▁▁▁▁▁▁
bleu,0.21449
loss,1.80683
lr_0,0.0
moving_avg_loss,1.8434
